In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Unzip Images
import zipfile
with zipfile.ZipFile('../input/leaf-classification/images.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
#Unzip Train Labels of species
import zipfile
with zipfile.ZipFile('../input/leaf-classification/train.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
# import zipfile
# with zipfile.ZipFile('../input/leaf-classification/sample_submission.csv.zip', 'r') as zip_ref:
#     zip_ref.extractall('./')

In [ ]:
# Import usefull lib's
# import os
# import pandas as pd
# import numpy as np
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [ ]:
#Making Dir for storage
os.mkdir('./Train')
os.mkdir('./Test')
os.mkdir('./Valid')

In [ ]:
#Length of images
len(os.listdir('./images'))

In [ ]:
#Reading train data to dataframe
df=pd.read_csv('./train.csv',index_col=False)

In [ ]:
#sample_sub_df=pd.read_csv('./sample_submission.csv',index_col=False)

In [ ]:
#sample_sub_df.head()

In [ ]:
#Length of train data frame
len(df)

In [ ]:
#No:of unique species
df['species'].nunique()

In [ ]:
# Creating a data frame to extract Image Id 
Species_Image = pd.DataFrame({'Id': range(1,991),
                       'Species': df.species})

In [ ]:
Species_Image#.copy()#.Species.nunique()

In [ ]:
#Seperating Images to train and test
for image_filename in os.listdir('./images'):
    a = int(image_filename.split('.')[0])
    if a > 990:
        imgsav = imread('./images'+'/'+image_filename)
        path = './Test'
        cv2.imwrite(os.path.join(path , image_filename), imgsav)
    else:
        imgsav = imread('./images'+'/'+image_filename)
        path = './Train'
        cv2.imwrite(os.path.join(path , image_filename), imgsav)

In [ ]:
#Check the length of train Images
len(os.listdir('./Train'))

In [ ]:
#Check the length of test Images
len(os.listdir('./Test'))

In [ ]:
#Import ImageDataGenerator from tesorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Generating Images to Train and validate
image_gen_one_v = ImageDataGenerator()#(vertical_flip=True)
image_gen_one_h = ImageDataGenerator()#(horizontal_flip=True)
image_gen_one_30d = ImageDataGenerator()#(rotation_range=30,fill_mode="constant")

In [ ]:
#Generating and saving images to train folder.
count=991
for i in range(1,991):
    image_filename=str(i)+'.jpg'
    leaf_img_path = './Train'+'/'+image_filename
    leaf_img = imread(leaf_img_path)
    imgsav=image_gen_one_v.random_transform(leaf_img)
    path= './Train'
    image_filename=str(count)+".jpg"
    cv2.imwrite(os.path.join(path , image_filename), imgsav)
    count=count+1

In [ ]:
#Generating and saving images to train folder.
count=1981
for i in range(1,991):
    image_filename=str(i)+'.jpg'
    leaf_img_path = './Train'+'/'+image_filename
    leaf_img = imread(leaf_img_path)
    
    imgsav=image_gen_one_h.random_transform(leaf_img)
    path= './Train'
    image_filename=str(count)+".jpg"
    cv2.imwrite(os.path.join(path , image_filename), imgsav)
    count=count+1

In [ ]:
#Generating and saving images to Valid folder.
#count=1981
for i in range(1,991):
    image_filename=str(i)+'.jpg'
    leaf_img_path = './Train'+'/'+image_filename
    leaf_img = imread(leaf_img_path)
    imgsav=image_gen_one_30d.random_transform(leaf_img)
    path= './Valid'
    image_filename=str(i)+".jpg"
    cv2.imwrite(os.path.join(path , image_filename), imgsav)
    #count=count+1

In [ ]:
#Total length of Images in train folder
len(os.listdir('./Train'))

In [ ]:
#Values Created for first df to concatinate with main data frame (ImageID_Species)
Species_Image_v = pd.DataFrame({'Id': range(991,1981),
                       'Species': df.species})

In [ ]:
#Values Created for second df to concatinate with main data frame (ImageID_Species)
Species_Image_h = pd.DataFrame({'Id': range(1981,2971),
                       'Species': df.species})

In [ ]:
#Total Id and Species for for Train folder images. 
ImageId_Species = pd.concat([Species_Image,Species_Image_v,Species_Image_h])

In [ ]:
#Assinging image name to the ID columns as the folder image are of this format
ImageId_Species['Id']=ImageId_Species['Id'].astype(str)+".jpg"

In [ ]:
#Assinging ID column as index with split.
ImageId_Species['Index']=ImageId_Species['Id'].apply(lambda a: a.split('.')[0])

In [ ]:
#Seting up index starting with 1
ImageId_Species.set_index('Index', inplace=True)

In [ ]:
#Setting up Taind and Validation dataframe
y_Train = ImageId_Species#.iloc[:2376,:]
#y_Test = ImageId_Species.iloc[2376:,:]
Validation_30deg = ImageId_Species.iloc[:990,:]

In [ ]:
y_Train


In [ ]:
Validation_30deg

In [ ]:
#Checking if both the train and valid have same number for species
y_Train.Species.nunique()

In [ ]:
#Checking if both the train and valid have same number for species
Validation_30deg.Species.nunique()

In [ ]:
datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
# Implementing learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [ ]:
image_shape =(256,256,1)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(99, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
train_generator = datagen.flow_from_dataframe(dataframe=y_Train,
                                              directory='./Train',
                                              x_col="Id", y_col="Species",
                                              color_mode="grayscale",
                                              class_mode="categorical", 
                                              target_size=(256,256), batch_size=16)

In [ ]:
test_generator = datagen.flow_from_dataframe(dataframe=Validation_30deg,
                                              directory='./Valid',
                                              x_col="Id", y_col="Species",
                                              color_mode="grayscale",
                                              class_mode="categorical", 
                                              target_size=(256,256), batch_size=16)

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=test_generator.n//test_generator.batch_size

In [ ]:
results = model.fit_generator(generator=train_generator,
                              steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=test_generator,
                              validation_steps=STEP_SIZE_VALID,
                              epochs=10)

In [ ]:

#Retriving Loss from model history
losses = pd.DataFrame(results.history)


In [ ]:
losses.head()

In [ ]:
#Plotting  Accutacy vs Val_accuracy
losses[['accuracy','val_accuracy']].plot()

In [ ]:
#Ploting Loss Vs Val_Loss
losses[['loss','val_loss']].plot()

In [ ]:
from PIL import Image, ImageFilter
import tensorflow as tf
import keras.preprocessing

In [ ]:
#Testing a image
image = Image.open('./Train/2970.jpg')
image = image.resize((256,256))
input_arr = keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])  # Convert single image to a batch.
input_arr.shape
predictions = model.predict(input_arr)
predictions

In [ ]:
label_map = (train_generator.class_indices)

In [ ]:
label_map

In [ ]:
#Prediction of all images and store it to a string.
L=[]
for i in range(991,1585):
    image_filename=str(i)+'.jpg'
    leaf_img_path = './Test'+'/'+image_filename
    leaf_img = Image.open(leaf_img_path)
    image = leaf_img.resize((256,256))
    input_arr = keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    #input_arr.shape
    predictions = model.predict(input_arr)
    Pre_Species = pd.DataFrame({'Id': list(predictions.reshape(99)),'LabelMap': list(label_map)})
    L.append(str(Pre_Species[Pre_Species.Id==1.0].LabelMap).split()[1])
    

In [ ]:
len(L)

In [ ]:
#Generating df for predicted images.
ImageId_Species_test = pd.DataFrame({'Id': [str(a)+'.jpg' for a in range(991,1585)] ,
                       'Species': L})

In [ ]:
ImageId_Species_test

In [ ]:
#Compair Predicted Image and Actual Image
from keras.preprocessing.image import load_img
SampleCheck=ImageId_Species_test.sample()
im_check=str(SampleCheck.Id).split()[1]
name_check=str(SampleCheck.Species).split()[1]
img=load_img('./Test/'+im_check)
plt.subplot(1,2,1)
plt.imshow(img)
for k in range(0,2971):
        i , j = tuple(ImageId_Species.iloc[k])
        if j == name_check:
            break
img_T=load_img('./Train/'+i)
plt.subplot(1,2,2)
plt.imshow(img_T)